<a href="https://colab.research.google.com/github/Ivan-KKH/APAI4011_GP/blob/master/model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.5 MB/s eta 0:00:00


In [104]:
#import libraries
import numpy as np
import pandas as pd
from transformers import pipeline
from google.colab import files
import io
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import math

In [105]:
#import CSV file
uploaded = files.upload()


Saving amazon.csv to amazon (2).csv


In [106]:
#read csv file
df = pd.read_csv(io.BytesIO(uploaded['amazon.csv']))

#dfReview for review product name and product description
dfText = df[['review_title', 'review_content']]

#preprocess text data
for column in range(2):
    dfText.iloc[:,column] = dfText.iloc[:,column].apply(lambda x: x.lower())
    dfText.iloc[:,column] = dfText.iloc[:,column].apply((lambda x: re.sub(r"http\S+"," ",x)))
    dfText.iloc[:,column] = dfText.iloc[:,column].apply((lambda x: re.sub('[^a-zA-z0-9\s]'," ",x)))

#combine product_name and about_product
dfText['combined'] = dfText['review_title'] + ' ' + dfText['review_content']
dfText['combined']

<ipython-input-106-223550e0fe9d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfText.iloc[:,column] = dfText.iloc[:,column].apply(lambda x: x.lower())
<ipython-input-106-223550e0fe9d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfText.iloc[:,column] = dfText.iloc[:,column].apply((lambda x: re.sub(r"http\S+"," ",x)))
<ipython-input-106-223550e0fe9d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

0       satisfied charging is really fast value for mo...
1       a good braided cable for your type c device go...
2       good speed for earlier versions good product w...
3       good product good one nice really nice product...
4       as good as original decent good one for second...
                              ...                        
1460    received the product without spanner excellent...
1461    ok everything was good couldn t return bcoz i ...
1462    very good work but front melt after 2 month go...
1463    fan speed is slow good quality good product go...
1464    works perfect ok good product nice product  re...
Name: combined, Length: 1465, dtype: object

In [107]:
#preprocess numeric inputs
#rating count
df['rating_count'] = df['rating_count'].replace(',','',regex=True).astype('float64')
df['rating_count'] = df['rating_count'].fillna(0) #replace NaN values with 0

#actual price
df['actual_price'] = df['actual_price'].replace(',','',regex=True)
df['actual_price'] = df['actual_price'].replace('₹','',regex=True).astype('float64')
#discounted price
df['discounted_price'] = df['discounted_price'].replace(',','',regex=True)
df['discounted_price'] = df['discounted_price'].replace('₹','',regex=True).astype('float64')
#rating
df['rating'] = df['rating'].replace('|', '4.0').astype('float64')


In [108]:
#sentiment analysis
sentiment_pipeline = pipeline("sentiment-analysis")
sentiment = []
for review in tqdm(dfText['review_title']):
    #sentiment analysis using huggingface sentiment analyser
    review_sentiment = sentiment_pipeline(review[:512])
    #positive score if label='POSITIVE' else negative
    if review_sentiment[0]['label'] == 'POSITIVE':
        score = review_sentiment[0]['score']
    else:
        score = -1 * review_sentiment[0]['score']
    sentiment.append(score)
df['sentiment'] = sentiment

df[['review_title','sentiment']]

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
100%|██████████| 1465/1465 [02:19<00:00, 10.49it/s]


,review_title,sentiment
0,"Satisfied,Charging is really fast,Value for mo...",0.999341
1,"A Good Braided Cable for Your Type C Device,Go...",-0.610357
2,"Good speed for earlier versions,Good Product,W...",0.999560
3,"Good product,Good one,Nice,Really nice product...",0.995654
4,"As good as original,Decent,Good one for second...",0.999690
...,...,...
1460,"Received the product without spanner,Excellent...",0.976231
1461,"ok,everything was good couldn't return bcoz I ...",-0.954374
1462,"very good,Work but front melt after 2 month,Go...",-0.909177
1463,"Fan Speed is slow,Good quality,Good product,go...",0.997580


In [109]:
#Applying Preprocessing using Standard Scaler

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

scaler = StandardScaler()

#add review sentiment as an input variable
X = df[['actual_price', 'rating', 'rating_count', 'sentiment']]
y = df['discounted_price']

#normalise numeric inputs and output
X = scaler.fit_transform(X)
output_scaler = StandardScaler()
y = output_scaler.fit_transform(np.array(y).reshape(-1,1))

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 21, test_size = 0.3)

#fit multiple linear regression
regss = LinearRegression()
regss.fit(X_train, y_train)

#predict and print R2
y_pred = regss.predict(X_test)
print('R2 Score: ', r2_score(y_test, y_pred))

R2 Score:  0.9229431866536697


In [119]:
# make a prediction for an example of an out-of-sample observation
#Blink Outdoor (3rd Gen) - wireless, weather-resistant HD security camera, two-year battery life, motion detection, set up in minutes – 4 camera system
actual_price = 359.97
rating = 4.8
rating_count = 169381
#review = '4.0 out of 5 stars Still A Good Buy, Still A Bit Lacking...'
review = 'Works pretty good'

#sentiment analysis
rev_sent = sentiment_pipeline(review[:512])
#positive score if label='POSITIVE' else negative
if (rev_sent[0]['label'] == 'POSITIVE'):
    sentiment = review_sentiment[0]['score']
else:
    sentiment = -1 * review_sentiment[0]['score']

input = [[actual_price, rating, rating_count, sentiment]]
Xnew = scaler.transform(input)

# make a prediction
ynew = regss.predict(Xnew)

ynew = output_scaler.inverse_transform(ynew)

print(input)
print(f'suggested discounted price: {ynew[0][0]}')

[[359.97, 4.8, 169381, 0.9949361085891724]]
suggested discounted price: 226.44098983213007


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
